- Unable to extract borderless tables
- Unable to extract even bordered tables in correct format.
- The main purpose of this library is to just extract the text (native or non-native)

In [ ]:
from unstract.llmwhisperer import LLMWhispererClientV2
import pandas as pd
import os, re

In [2]:
pdf_path = "../Input/sample_table.pdf"
output_excel_path = "./Output/tables_output.xlsx"
os.makedirs(os.path.dirname(output_excel_path), exist_ok=True)

In [3]:
os.environ["LLMWHISPERER_API_KEY"] = "oediOQlucV0m9iebZEhW95LufbCdzBFXlBETlS1qdVU"
os.environ["LLMWHISPERER_BASE_URL"] = "https://llmwhisperer-api.us-central.unstract.com/api/v2"

In [4]:
client = LLMWhispererClientV2()
usage_info = client.get_usage_info()
print(usage_info)

2025-06-24 10:33:48,999 - unstract.llmwhisperer.client_v2 - DEBUG - logging_level set to DEBUG
2025-06-24 10:33:49,000 - unstract.llmwhisperer.client_v2 - DEBUG - base_url set to https://llmwhisperer-api.us-central.unstract.com/api/v2
2025-06-24 10:33:49,002 - unstract.llmwhisperer.client_v2 - DEBUG - get_usage_info called
2025-06-24 10:33:49,003 - unstract.llmwhisperer.client_v2 - DEBUG - url: https://llmwhisperer-api.us-central.unstract.com/api/v2/get-usage-info


{'current_page_count': -1, 'daily_quota': 100, 'monthly_quota': -1, 'overage_page_count': -1, 'subscription_plan': 'LLM Whisperer Free', 'today_page_count': 29}


In [5]:
whisper = client.whisper(
    file_path=pdf_path, 
    wait_for_completion=True,
    mark_vertical_lines=True,      # reproduce vertical lines
    mark_horizontal_lines=True,    # reproduce horizontal lines
    wait_timeout=200
)

result = whisper["extraction"]
print(result['result_text'])

2025-06-24 10:33:52,511 - unstract.llmwhisperer.client_v2 - DEBUG - whisper called
2025-06-24 10:33:52,513 - unstract.llmwhisperer.client_v2 - DEBUG - api_url: https://llmwhisperer-api.us-central.unstract.com/api/v2/whisper
2025-06-24 10:33:52,515 - unstract.llmwhisperer.client_v2 - DEBUG - params: {'mode': 'form', 'output_mode': 'layout_preserving', 'page_seperator': '<<<', 'pages_to_extract': '', 'median_filter_size': 0, 'gaussian_blur_radius': 0, 'line_splitter_tolerance': 0.4, 'horizontal_stretch_factor': 1.0, 'mark_vertical_lines': True, 'mark_horizontal_lines': True, 'line_spitter_strategy': 'left-priority', 'add_line_nos': False, 'lang': 'eng', 'tag': 'default', 'filename': '', 'webhook_metadata': '', 'use_webhook': ''}
2025-06-24 10:33:53,902 - unstract.llmwhisperer.client_v2 - DEBUG - whisper_status called
2025-06-24 10:33:53,904 - unstract.llmwhisperer.client_v2 - DEBUG - url: https://llmwhisperer-api.us-central.unstract.com/api/v2/whisper-status
2025-06-24 10:33:54,452 - uns



Example table 

This is an example of a data table. 

     ------------------------------------------------------------------------
     |               |              |             |                | Results 
     |               |              |             | Ballots 
     | Disability    |              | Ballots 
                                                                    ----------------
     |               | Participants |             | Incomplete/ Accuracy   Time to 
     | Category      |              | Completed 
     |               |              |             | Terminated 
     |               |              |             |                |       complete 

     -------------------------------------------------------------------------------
     | Blind         |  5         1 |           4                   | 34.5%, n=1 1199 sec, n=1 

     -------------------------------------------------------------------------------
     | Low Vision    |  5         2 |        

In [6]:
text = result['result_text']
lines = [l for l in text.splitlines() if l.strip()]

# 1. Remove pure-dash separator lines (allow leading spaces)
content_lines = [l for l in lines if not re.match(r'^\s*-+\s*$', l)]

# 2. Collect every line that contains at least one pipe
table_lines = [l for l in content_lines if '|' in l]

# 3. Split each line into cells, padding so each row has same width
rows = []
for l in table_lines:
    # strip just the leading/trailing whitespace (but keep pipes)
    row = l.strip()
    # ensure it both starts and ends with '|' so split is consistent
    if not row.startswith('|'):
        row = '|' + row
    if not row.endswith('|'):
        row = row + '|'
    # split on '|' and drop the empty first/last
    cells = [c.strip() for c in row.split('|')[1:-1]]
    rows.append(cells)

if not rows:
    print("❌ No pipe-delimited lines found at all.")
else:
    # 4. Pad rows so each has the same number of columns
    max_cols = max(len(r) for r in rows)
    padded = [r + ['']*(max_cols - len(r)) for r in rows]

    # 5. Build a DataFrame (generic column names)
    cols = [f"Col{i+1}" for i in range(max_cols)]
    df = pd.DataFrame(padded, columns=cols)
    print(df)

    # 6. Write to Excel
    writer = pd.ExcelWriter(output_excel_path, engine="xlsxwriter")
    df.to_excel(writer, sheet_name="ExtractedTable", index=False)
    writer.close()
    print(f"✅ Wrote {len(df)} rows × {len(cols)} columns to excel file: {output_excel_path}")


          Col1          Col2                              Col3  \
0                                                                
1                                                                
2   Disability                                         Ballots   
3               Participants                                     
4     Category                                       Completed   
5                                                                
6                                                                
7        Blind   5         1                                 4   
8   Low Vision   5         2  3      98.3% n=2   1716 sec, n=3   
9                                                                
10   Dexterity   5         4                                 1   
11    Mobility   3         3                                 0   

                              Col4             Col5             Col6  
0                                           Results                   